In [ ]:
from pathlib import Path
import sys
import datetime as dt
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates

path_to_repo = '../../'
path_to_repo = Path(path_to_repo).resolve()
path_to_src = path_to_repo / 'ext'
sys.path.append(str(path_to_src.absolute()))

from exp_processing.log_processor import LogProcessor

In [ ]:
case_name = '20250410'
#path_to_examples = '..'
path_to_examples = path_to_repo / 'data' / 'Shared_Data_notinGIT' / 'VM Results'

logs = LogProcessor(case_name, path_to_examples)

In [ ]:
rt_df = logs.parse_common_log()

In [ ]:
da_df = logs.parse_DA_data()

In [ ]:
det_df = logs.parse_DET_data()

# Day Ahead

In [ ]:
# Create a start and end time for plots 
PltStrtTime = da_df.index.get_level_values(1)[0]
PltEndTime = da_df.index.get_level_values(1)[-1]
print(f'Day Ahead Optimization went from {PltStrtTime} to {PltEndTime}')

In [ ]:

feeder_base_load = 3466000/1000 # kW

load_profile = logs.load_loadprofile(feeder_base_load)

#load_profile = pd.read_csv(load_profile_fn, header=None, names=['time info', 'load scale'])
#load_profile['timestamp'] = pd.date_range(start=PltStrtTime, periods=len(load_profile), freq=pd.Timedelta(load_profile['time info'][1]))
#load_profile = load_profile.set_index('timestamp')
#load_profile['load kW'] = load_profile['load scale'] * feeder_base_load

In [ ]:
subs_p0_rup = da_df.loc[1]['Subs_p0'] - da_df.loc[1]['Subs_rup']
subs_p0_rdn = da_df.loc[1]['Subs_p0'] + da_df.loc[1]['Subs_rdn']

fig, axes = plt.subplots(2, figsize = (10,6), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
color = 'tab:blue'
axes[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
axes[0].step(da_df.loc[1].index,da_df.loc[1]['Subs_p0'], color = color, where='post',label='Power')
axes[0].fill_between(da_df.loc[1].index,subs_p0_rup,subs_p0_rdn,color = color, step='post',alpha=0.5,label='Reserves')
axes[0].legend()
axes[0].set(ylabel='Power [kW]', title = 'Substation Power with Reserves')


axes[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
color = 'tab:green'
axes[1].step(da_df.loc[1].index,da_df.loc[1]['DA ISO Rup Price'], color = color, alpha = 0.6, linestyle = '-.', where='post',label='Reserve Up')
axes[1].step(da_df.loc[1].index,da_df.loc[1]['DA ISO Rdn Price'], color = color, alpha = 0.3, linestyle = '--', where='post',label='Reserve Dn')
axes[1].step(da_df.loc[1].index,da_df.loc[1]['DA ISO Energy Price'], color = color, where='post',label='System')
axes[1].set(ylabel='$/kWh', title = 'Day Ahead Prices')
axes[1].legend()

axes[1].set(xlabel = 'Time of Day')
axes[1].xaxis.set_minor_locator(dates.HourLocator(interval=1))   # every 2 hours
axes[1].xaxis.set_major_locator(dates.HourLocator(byhour=range(1,24,3)))    # No day info
axes[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

logs.add_to_export_list(fig)



In [ ]:
total_load = [sum(sum(logs.fcasts_data['Loads'][load]['PL']['xbar'][time][phase] /1000 for phase in logs.DA_index['phases']) for load in logs.fcasts_data['Loads'].keys()) for time in range(len(logs.DA_index['frames']))]
sigma = [np.sqrt(sum(sum((logs.fcasts_data['Loads'][load]['PL']['sigma'][time][phase] /1000)**2 for phase in logs.DA_index['phases']) for load in logs.fcasts_data['Loads'].keys())) for time in range(len(logs.DA_index['frames']))]
total_load_up = [total_load[i] + 2*sigma[i] for i in range(len(logs.DA_index['frames']))]
total_load_dn = [total_load[i] - 2*sigma[i] for i in range(len(logs.DA_index['frames']))]

# duplicating the last value to make the plot look better
total_load.append(total_load[-1])
total_load_up.append(total_load_up[-1])
total_load_dn.append(total_load_dn[-1])
sigma.append(sigma[-1])

df = pd.DataFrame(index=da_df.loc[1].index)
df['max_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').max()
df['min_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').min()

def reverse_quantile(value, distribution):
    return (distribution<value).mean()*100

for index, time in enumerate(da_df.loc[1].index):
    distribution = np.random.normal(total_load[index], sigma[index], 100000)
    df.loc[time, 'max_load_percentile'] = reverse_quantile(df.loc[time, 'max_load_fcast'], distribution)
    df.loc[time, 'min_load_percentile'] = reverse_quantile(df.loc[time, 'min_load_fcast'], distribution)
df['coverage'] = df['max_load_percentile'] - df['min_load_percentile']

fig, axes = plt.subplots(2, figsize = (10,6), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
color = 'tab:blue'
axes[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
axes[0].fill_between(da_df.loc[1].index,total_load_up,total_load_dn,color='tab:grey', step='post',alpha=0.2,label='Forecast Uncertainty (2$\sigma$)')
axes[0].fill_between(df.index,df['max_load_fcast'].astype('float'),df['min_load_fcast'].astype('float'),color='tab:blue', step='post',alpha=0.15,label='Load Profiles across Scenarios')
for scenario in da_df.index.levels[0]:
    axes[0].step(da_df.loc[scenario].index,da_df.loc[scenario]['total_load'], where='post',color = color, alpha = 0.3, linestyle = '--')
axes[0].step(da_df.loc[1].index,da_df.loc[1]['total_load'], where='post',color = color, alpha = 0.3, linestyle = '--', label='Selected Scenarios')
axes[0].step(da_df.loc[1].index,total_load, where='post', color='tab:grey',label='Forecast Average Power')
axes[0].legend()
axes[0].set(ylabel='Power [kW]', title = 'Load profiles across Scenarios')

axes[1].step(df.index,df['max_load_percentile'],label='Max Load Percentile', where='post', color='tab:blue')
axes[1].step(df.index,df['min_load_percentile'],label='Min Load Percentile', where='post', color='tab:blue')
axes[1].set(xlabel = 'Time of Day', ylabel='Percentile [%]')

from matplotlib.colors import LinearSegmentedColormap
c = ["darkred","red","lightcoral", "palegreen","green","darkgreen"]
v = [0,.15,.4,0.6,.9,1.]
l = list(zip(v,c))
cmap=LinearSegmentedColormap.from_list('rg',l, N=256)
xlim = axes[1].get_xlim()
ylim = axes[1].get_ylim()
polygon = axes[1].fill_between(df.index, df['max_load_percentile'], df['min_load_percentile'], color='none', lw=0, step='post')
verts = np.vstack([p.vertices for p in polygon.get_paths()])
filling = axes[1].imshow(df['coverage'][:-1].to_numpy().reshape(1, -1), cmap=cmap, aspect='auto',
                    extent=[verts[:, 0].min(), verts[:, 0].max(), verts[:, 1].min(), verts[:, 1].max()])
filling.set_clip_path(polygon.get_paths()[0], transform=axes[1].transData)
axes[1].set_xlim(xlim) # the limits need to be set again because imshow sets zero margins
axes[1].set_ylim(ylim)
for i, v in enumerate(df['coverage'][:-1]):
    axes[1].text(df.index[i]+dt.timedelta(minutes=30), 40, f'{v:.0f}', ha="center")


axes[1].xaxis.set_minor_locator(dates.HourLocator(interval=1))   # every 1 hours
axes[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
axes[1].xaxis.set_major_locator(dates.HourLocator(byhour=range(1,24,3)))    # No day info

logs.add_to_export_list(fig)

In [ ]:
for ind, PV in enumerate(logs.PVs):
    fig, ax = plt.subplots(figsize = (10,6))
    ax.grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    # Confidence Interval
    pv_p = [logs.fcasts_data['DERs'][PV]['Pmax']['xbar'][time]/1000 for time in range(len(logs.DA_index['frames']))]
    pv_p_sigma = [logs.fcasts_data['DERs'][PV]['Pmax']['sigma'][time]/1000 for time in range(len(logs.DA_index['frames']))]
    pv_p_up = [pv_p[i] + 2*pv_p_sigma[i] for i in range(len(logs.DA_index['frames']))]
    pv_p_dn = [pv_p[i] - 2*pv_p_sigma[i] for i in range(len(logs.DA_index['frames']))]
    pv_p.append(pv_p[-1])
    pv_p_up.append(pv_p_up[-1])
    pv_p_dn.append(pv_p_dn[-1])
    
    ax.fill_between(da_df.loc[1].index,pv_p_up,pv_p_dn,step='post',color = f'C{ind}', alpha=0.1,label='Forecast Uncertainty (2$\sigma$)')
    for scenario in logs.DA_index['scenarios']:
        ax.step(da_df.loc[scenario].index,da_df.loc[scenario][f'{PV}_Pmax'], where='post',color = f'C{ind}', alpha = 0.3, linestyle = '--')
    ax.step(da_df.loc[1].index,da_df.loc[1][f'{PV}_Pmax'], where='post',label='Selected Scenarios', color = f'C{ind}', alpha = 0.3, linestyle = '--')
    ax.step(da_df.loc[1].index,da_df.loc[1][f'{PV}_Pmax'], color='black', where='post',label='Forecast Average Power')
    
    ax.legend()
    ax.set(xlabel = 'Time of Day', ylabel='Power [kW]', title = f'{PV} Power Forecast')

    ax.xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax.xaxis.set_major_locator(plt.NullLocator())    # No day info


In [ ]:
fig, ax = plt.subplots(figsize = (10,6))

ax.step(da_df.loc[1].index,da_df.loc[1][[f'{pv}_Pmax' for pv in logs.PVs]].sum(axis=1), where='post', color='grey', linestyle='-.',label=f'Total PV max')
ax.step(da_df.loc[1].index,da_df.loc[1][[f'{pv}_P' for pv in logs.PVs]].sum(axis=1), where='post', label='Total PV')

ax.legend()
ax.set(xlabel = 'Time of Day', ylabel='Power [kW]', title = 'Total PV Power')

ax.xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
ax.xaxis.set_major_locator(plt.NullLocator())    # No day info

In [ ]:
for ind, PV in enumerate(logs.PVs):
    fig, ax = plt.subplots(figsize = (10,6))
    for scenario in da_df.index.levels[0]:
        ax.step(da_df.loc[scenario].index,da_df.loc[scenario][f'{PV}_P'], where='post',color = f'C{ind}', alpha = 0.3)
    ax.step(da_df.loc[1].index,da_df.loc[1][f'{PV}_P'], color='black', where='post',label='Power')
    
    
    ax.legend()
    ax.set(xlabel = 'Time of Day', ylabel='Power [kW]', title = f'{PV} Power')

    ax.xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax.xaxis.set_major_locator(plt.NullLocator())    # No day info



In [ ]:
for ind, PV in enumerate(logs.PVs):
    fig, ax = plt.subplots(figsize = (10,6))

    for scenario in da_df.index.levels[0]:
        pv_curtail = 100*(da_df.loc[scenario][f'{PV}_Pmax'] - da_df.loc[scenario][f'{PV}_P'])/da_df.loc[scenario][f'{PV}_Pmax']
        ax.step(da_df.loc[scenario].index,pv_curtail, where='post',color = f'C{ind}', alpha = 0.3)
    ax.step(da_df.loc[1].index,100*(da_df.loc[1][f'{PV}_Pmax'] - da_df.loc[1][f'{PV}_P'])/da_df.loc[1][f'{PV}_Pmax'], color='black', where='post',label='Power')
    
    ax.legend()
    ax.set(xlabel = 'Time of Day', ylabel='Percentage of Pmax [%]', title = f'{PV} Curtailment')

    ax.xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax.xaxis.set_major_locator(plt.NullLocator())    # No day info


In [ ]:
for ind, PV in enumerate(logs.PVs):
    fig, ax = plt.subplots(figsize = (10,6))
    test = da_df.pivot_table(index = 'scenario', columns='time' , values = [f'{PV}_P',f'{PV}_Pmax'])
    curtail=100*(test[f'{PV}_Pmax'] - test[f'{PV}_P'])/test[f'{PV}_Pmax']

    im = ax.pcolormesh(curtail, cmap='viridis', vmin=0, vmax=100)
    
    ax.set(xlabel = 'Time of Day', ylabel='Scenario', title = f'{PV} Curtailment')
    fig.colorbar(im)
    #ax.xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    # ax.xaxis.set_major_locator(plt.NullLocator())    # No day info

In [ ]:

fig, ax = plt.subplots(figsize = (10,6))
for batt in logs.BATTs:
    ax.step(da_df.loc[1].index,da_df.loc[1][f'{batt}_P'], where='post',label=batt)


ax.legend()
ax.set(xlabel = 'Time of Day', ylabel='Power [kW]', title = 'Battery Power')

ax.xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
ax.xaxis.set_major_locator(plt.NullLocator())    # No day info

In [ ]:

fig, ax = plt.subplots(figsize = (10,6))
all_batts = sum([da_df.loc[1][f'{batt}_P'] for batt in logs.BATTs])
ax.step(da_df.loc[1].index,all_batts, where='post',label='Sum of Battery Power')


ax.legend()
ax.set(xlabel = 'Time of Day', ylabel='Power [kW]', title = 'Battery Power')

ax.xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
ax.xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
ax.xaxis.set_major_locator(plt.NullLocator())    # No day info

In [ ]:
fig, ax = plt.subplots(3,figsize = (10,9),sharex=True)

for batt in logs.BATTs:
    ax[0].plot(da_df.loc[1].index,da_df.loc[1][f'{batt}_E'],label=batt)

ax[0].legend()
ax[0].set(ylabel='Energy [kWh]', title = 'Individual Battery Energy')

total_soc = 0
for batt in logs.BATTs:
    ## SOC min at 10%
    E_min = logs.static_data['DERs'][batt]['emax']*0.1/1000
    ## SOC max at 90%
    E_max = logs.static_data['DERs'][batt]['emax']*0.9/1000 
    batt_soc = 100/(E_max-E_min)*da_df.loc[1][f'{batt}_E']-100*E_min/(E_max-E_min)
    total_soc += batt_soc
    ax[1].plot(da_df.loc[1].index,batt_soc,label=batt)
ax[1].set(ylabel='SoC [%]', title = 'Individual Battery SoC')

ax[2].plot(da_df.loc[1].index,total_soc/len(logs.BATTs),label='Total SOC')
ax[2].legend()
ax[2].set(xlabel = 'Time of Day', ylabel='SoC [%]', title = 'Total Battery SoC')

ax[2].xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
ax[2].xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
ax[2].xaxis.set_major_locator(plt.NullLocator())    # No day info

In [ ]:
for ind, batt in enumerate(logs.BATTs):
    
    # Batt data
    x0 = da_df.loc[1].index[0]
    x1 = da_df.loc[1].index[-1]
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][batt]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][batt]['smax']/1000
    except:
        P_min = -P_max

    ## SOC min at 10%
    E_min = logs.static_data['DERs'][batt]['emax']*0.1/1000
    ## SOC max at 90%
    E_max = logs.static_data['DERs'][batt]['emax']*0.9/1000

    fig, ax = plt.subplots(2, figsize = (10,6), sharex=True)
    ax[0].hlines(P_max,x0,x1,linestyles='dashed', color = 'grey')
    ax[0].hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    for scenario in da_df.index.levels[0]:
        ax[0].step(da_df.loc[scenario].index,da_df.loc[scenario][f'{batt}_P'], where='post',color = f'C{ind}', alpha = 0.3)
    ax[0].step(da_df.loc[1].index,da_df.loc[1][f'{batt}_P'], color='black', where='post',label='Power')
    ax[0].legend()
    ax[0].set(ylabel='Power [kW]', title = f'Battery Power of {batt}')
    ax[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

    ax[1].hlines(E_max,x0,x1,linestyles='dashed',label='Emax', color = 'grey')
    ax[1].hlines(E_min,x0,x1,linestyles='dashed',label='Emin', color = 'grey')
    for scenario in da_df.index.levels[0]:
        ax[1].plot(da_df.loc[scenario].index,da_df.loc[scenario][f'{batt}_E'],color = f'C{ind}', alpha = 0.3)
    ax[1].plot(da_df.loc[1].index,da_df.loc[1][f'{batt}_E'], color='black',label='Power')
    ax[1].set(xlabel = 'Time of Day', ylabel='Energy [kWh]', title = f'Battery Energy of {batt}')
    ax[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax[1].xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
    ax[1].xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax[1].xaxis.set_major_locator(plt.NullLocator())    # No day info
    

In [ ]:
for ind, batt in enumerate(logs.BATTs):
    
    # Batt data
    x0 = da_df.loc[1].index[0]
    x1 = da_df.loc[1].index[-1]
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][batt]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][batt]['smax']/1000
    except:
        P_min = -P_max

    ## SOC min at 10%
    E_min = logs.static_data['DERs'][batt]['emax']*0.1/1000
    ## SOC max at 90%
    E_max = logs.static_data['DERs'][batt]['emax']*0.9/1000

    batt_p_rup = da_df.loc[1][f'{batt}_P'] + da_df.loc[1][f'{batt}_Rup']
    batt_p_rdn = da_df.loc[1][f'{batt}_P'] - da_df.loc[1][f'{batt}_Rdn']

    fig, ax = plt.subplots(2, figsize = (10,6), sharex=True)
    ax[0].hlines(P_max,x0,x1,linestyles='dashed', color = 'grey')
    ax[0].hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    ax[0].step(da_df.loc[1].index,da_df.loc[1][f'{batt}_P'], where='post',label=batt, color = f'C{ind}')
    ax[0].fill_between(da_df.loc[1].index,batt_p_rup,batt_p_rdn,step='post',color = f'C{ind}', alpha=0.3,label='Reserves')
    ax[0].legend()
    ax[0].set(ylabel='Power [kW]', title = f'Battery Power of {batt}')
    ax[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

    ax[1].hlines(E_max,x0,x1,linestyles='dashed',label='Emax', color = 'grey')
    ax[1].hlines(E_min,x0,x1,linestyles='dashed',label='Emin', color = 'grey')
    ax[1].plot(da_df.loc[1].index,da_df.loc[1][f'{batt}_E'],label=batt, color = f'C{ind}')
    ax[1].set(xlabel = 'Time of Day', ylabel='Energy [kWh]', title = f'Battery Energy of {batt}')
    ax[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax[1].xaxis.set_minor_locator(dates.HourLocator(interval=2))   # every 2 hours
    ax[1].xaxis.set_minor_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax[1].xaxis.set_major_locator(plt.NullLocator())    # No day info

# Entire Simulation

In [ ]:
# Create a start and end time for plots (Start at beginning and end 4 hours later)
PltStrtTime = rt_df.index[0]
PltEndTime = min(rt_df.index[-1], PltStrtTime + dt.timedelta(hours = 4))
print(f'The simulation went from {PltStrtTime} to {PltEndTime}')

In [ ]:
#plot the total dispatch and the substation power vs. 
pltDF = pd.concat([rt_df[['Subs_p0_ISO','Subs_p0']], da_df.loc[1]['total_load']], axis = 1)
pltDF = pltDF.ffill()
pltDF.index = pltDF.index.tz_convert('US/Pacific')
pltDF = pltDF[PltStrtTime:PltEndTime]

plot = pltDF.plot(figsize = (10,6))

plot.set(xlabel = 'Day of Month and Time of Day', ylabel='Power [kW]', title = 'ISO Dispatch vs. Substation Power recorded in Simulation')
plot.grid()

In [ ]:
load_profile

In [ ]:
pltDF

In [ ]:
df = pd.DataFrame()
df['max_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').max()
df['min_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').min()


pltDF = pd.concat([rt_df[['Subs_p0_ISO','Subs_p0']], load_profile['load kW'], df['max_load_fcast'], df['min_load_fcast']], axis = 1)
pltDF = pltDF.ffill()
pltDF = pltDF[PltStrtTime:PltEndTime]

fig, ax = plt.subplots(1, figsize = (10,6), sharex=True)
#plot the total dispatch and the substation power vs. 
ax.fill_between(pltDF.index,pltDF['max_load_fcast'],pltDF['min_load_fcast'],step='post',alpha=0.1,label='Load Forecasts Range across Scenarios', color = 'tab:blue')
ax.step(pltDF.index,pltDF['load kW'], where='post',label='Actual Load', color = 'tab:blue')
ax.step(pltDF.index,pltDF['Subs_p0_ISO'], where='post',label='ISO Dispatch', color = 'tab:green')
ax.plot(pltDF.index,pltDF['Subs_p0'],label='Measured Substation Power', linestyle='--', color = 'tab:orange')

ax.legend()

ax.grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

ax.set(xlabel = 'Time of Day', ylabel='Power [kW]', title = 'ISO Dispatch vs. Substation Power recorded in Simulation')

ax.xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
ax.xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
ax.xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

In [ ]:
#plot the total dispatch and the substation power vs. 
pltDF = pd.concat([rt_df[['Subs_p0_ISO','Subs_p0', 'MPC_dispatch', 'total_dispatch', 'total_dispatch_corrected']]],axis =1)
pltDF = pltDF[PltStrtTime:PltEndTime]

plot = pltDF.plot(figsize = (10,6))
#plot = RT_DF['ISO_Dispatch'].plot()
plot.set(xlabel = 'Day of Month and Time of Day', ylabel='Power [kW]', title = 'Substation vs. DER Power output')
plot.grid()

In [ ]:
df = pd.DataFrame()
df['max_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').max()
df['min_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').min()

Relevant_RT  = rt_df[['Subs_p0_ISO','Subs_p0', 'MPC_dispatch', 'MPC_Rup', 'MPC_Rdn', 'total_dispatch', 'total_dispatch_corrected', 'Subs_p0_MPC']].copy()
Relevant_DA = da_df.loc[1][['total_load','Subs_p0']].copy()
Relevant_DA.rename(columns={'Subs_p0': 'DA_Subs_p0'}, inplace=True)
Relevant_DET = det_df[['Subs_p0','eta']].copy()
print(type(Relevant_DET))
Relevant_DET.rename(columns={'Subs_p0': 'DET_Subs_p0', 'eta':'DET_eta'}, inplace=True)
pltDF = pd.concat([Relevant_RT, load_profile['load kW'], Relevant_DET, Relevant_DA, df['max_load_fcast'], df['min_load_fcast']], axis = 1)
pltDF = pltDF.ffill()
pltDF = pltDF[PltStrtTime:PltEndTime]


fig, ax = plt.subplots(2, figsize = (10,10), sharex=True)
#plot the total dispatch and the substation power vs. 
ax[0].fill_between(pltDF.index,pltDF['max_load_fcast'],pltDF['min_load_fcast'],step='post',alpha=0.1,label='Load Forecasts Range across Scenarios', color = 'tab:blue')
#ax[0].step(pltDF.index,pltDF['Baseline_Subs_p0'], where='post',label='Actual Net Load', color = 'tab:blue')
ax[0].step(pltDF.index,pltDF['load kW'], where='post',label='Actual Simulation Load', color = 'tab:blue')
ax[0].step(pltDF.index,pltDF['Subs_p0_ISO'], where='post',label='ISO Dispatch', color = 'tab:red')
ax[0].plot(pltDF.index,pltDF['Subs_p0'],label='Measured Substation Power', linestyle='--', color = 'tab:orange')
ax[0].plot(pltDF.index,pltDF['DA_Subs_p0'],label='Day Ahead Substation Power Offer', color = 'tab:cyan')
ax[0].plot(pltDF.index,pltDF['DET_Subs_p0'],label='Post-Market DA Substation Power', color = 'tab:purple')
ax[0].plot(pltDF.index,pltDF['Subs_p0_MPC'],label='MPC Substation Power', linestyle=':', linewidth=3, color = 'tab:green')

ax[0].legend()
ax[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

ax[0].set (ylabel='Power [kW]', title = 'ISO Dispatch vs. Substation Power recorded in Simulation')

#plot the total dispatch and the substation power vs. 
ax[1].step(pltDF.index,pltDF['MPC_dispatch'], where='post',label='MPC Dispatch', color = 'tab:green')
ax[1].fill_between(pltDF.index,pltDF['MPC_dispatch']+pltDF['MPC_Rup'],pltDF['MPC_dispatch']-pltDF['MPC_Rdn'],step='post',alpha=0.1,label='Load Forecasts Range across Scenarios', color = 'tab:green')
ax[1].step(pltDF.index,pltDF['total_dispatch'], where='post',label='Actual Dispatch', color = 'tab:orange')
#ax[1].plot(pltDF.index,pltDF['total_dispatch_corrected'],label='Actual Dispatch Corrected', linestyle='--')

ax[1].legend()
ax[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

ax[1].set (xlabel ='Time of Day', ylabel='Power [kW]', title = 'DER Power output')

ax[1].xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
ax[1].xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
ax[1].xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
ax[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
logs.add_to_export_list(fig)

In [ ]:
#[print(k) for k in da_df.keys()]
da_df['eta']
#DA - Subs_p0, Subs_rup, Subs_rdn, eta
#DET - Subs_p0, Subs_rup, Subs_rdn, eta, eta_up_reserves, eta_dn_reserves
#RT - MPC_Rup, MPC_Rdn ,Subs_p0_MPC


In [ ]:
df = pd.DataFrame()
df['max_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').max()
df['min_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').min()


pltDF = pd.concat([rt_df[['Subs_p0_ISO','Subs_p0', 'MPC_dispatch', 'MPC_Rup', 'MPC_Rdn', 'total_dispatch', 'total_dispatch_corrected']], load_profile['load kW'], da_df.loc[1]['total_load'], df['max_load_fcast'], df['min_load_fcast']], axis = 1)
pltDF = pltDF.ffill()
pltDF = pltDF[PltStrtTime:PltEndTime]

fig, ax = plt.subplots(2, figsize = (10,10), sharex=True)
#plot the total dispatch and the substation power vs. 
ax[0].fill_between(pltDF.index,pltDF['max_load_fcast'],pltDF['min_load_fcast'],step='post',alpha=0.1,label='Load Forecasts Range across Scenarios', color = 'tab:blue')
#ax[0].step(pltDF.index,pltDF['Baseline_Subs_p0'], where='post',label='Actual Net Load', color = 'tab:blue')
ax[0].step(pltDF.index,pltDF['load kW'], where='post',label='Actual Simulation Load', color = 'tab:blue')
ax[0].step(pltDF.index,pltDF['Subs_p0_ISO'], where='post',label='ISO Dispatch', color = 'tab:red')
ax[0].plot(pltDF.index,pltDF['Subs_p0'],label='Measured Substation Power', linestyle='--', color = 'tab:orange')

ax[0].legend()
ax[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

ax[0].set (ylabel='Power [kW]', title = 'ISO Dispatch vs. Substation Power recorded in Simulation')

#plot the total dispatch and the substation power vs. 
ax[1].step(pltDF.index,pltDF['MPC_dispatch'], where='post',label='MPC Dispatch', color = 'tab:green')
ax[1].fill_between(pltDF.index,pltDF['MPC_dispatch']+pltDF['MPC_Rup'],pltDF['MPC_dispatch']-pltDF['MPC_Rdn'],step='post',alpha=0.1,label='Load Forecasts Range across Scenarios', color = 'tab:green')
ax[1].step(pltDF.index,pltDF['total_dispatch'], where='post',label='Actual Dispatch', color = 'tab:orange')
#ax[1].plot(pltDF.index,pltDF['total_dispatch_corrected'],label='Actual Dispatch Corrected', linestyle='--')

ax[1].legend()
ax[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

ax[1].set (xlabel ='Time of Day', ylabel='Power [kW]', title = 'DER Power output')

ax[1].xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
ax[1].xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
ax[1].xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
ax[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
logs.add_to_export_list(fig)

## TMM Figures / Analysis

In [ ]:
use_TMM = logs.config['use_tmm']

In [ ]:
#Check prices are consistent across optimizations:
da_stuff = da_df.loc[1]['DA ISO Energy Price'].copy().rename('DA_ISOEnergyPrice')
pltDF = pd.concat([da_stuff, det_df['DA ISO Energy Price'].rename('DET_ISOEnergyPrice'), rt_df['DA ISO Energy Price'].rename('RT_ISOEnergyPrice')], axis = 1)
pltDF = pltDF.ffill()
pltDF = pltDF[PltStrtTime:PltEndTime]
pltDF.plot()

In [ ]:
if use_TMM:
    pltDF = rt_df[['RT ISO Energy Price', 'DA ISO Energy Price', 'TMM_1_price']]
    pltDF = pltDF[PltStrtTime:PltEndTime]
    pltDF.plot()

In [ ]:
if use_TMM:
    #A plot to see if the house is following the setpoint
    All_houses = [f'n{num+1}' for num in range(40)]
    keys = rt_df.keys().to_list()

    #Batt_list = ['n2', 'n3', 'n4', 'n5', 'n6', 'n16', 'n25', 'n9', 'n11', 'n31', 'n13', 'n14', 'n15', 'n23', 'n37', 'n28', 'n29', 'n30', 'n22', 'n38'] 
    #Non_list = ['n1', 'n2', 'n3', 'n4', 'n5', 'n7', 'n9', 'n13', 'n14', 'n18', 'n19', 'n24', 'n26', 'n29', 'n31', 'n34']
    #indexer = [ house not in Non_list for house in All_houses ]

    indexer = [ f'{house}T_set' in keys for house in All_houses ]
    Houses_HVAC = [house for house, index in zip(All_houses,indexer) if index]

    indexer = [ f'{house}_Batt_set_command' in keys for house in All_houses ]
    Batt_list = [house for house, index in zip(All_houses,indexer) if index]

    indexer = [house in Batt_list for house in Houses_HVAC]
    Houses_Batt_HVAC = [house for house, index in zip(Houses_HVAC,indexer) if index]

    indexer = [house not in Batt_list for house in Houses_HVAC]
    Houses_just_HVAC = [house for house, index in zip(Houses_HVAC,indexer) if index]

    print(f'The houses that have Coooling: {Houses_HVAC}')
    print(f'The houses that have Coooling without a Battery: {Houses_just_HVAC}')
    print(f'The houses that have both a Battery and Coooling: {Houses_Batt_HVAC}')

    house = 'n22' 
    pltDF = rt_df[[f'{house}T_set',f'{house}T_set_command']]
    pltDF = pltDF[PltStrtTime:PltEndTime]
    if house in Houses_Batt_HVAC:
        pltDF2 = rt_df[[f'{house}_total_power',f'{house}_Batt_set_command']]
        pltDF3 = rt_df[f'{house}_total_power']-rt_df[f'{house}_Batt_set_command']-rt_df[f'{house}_unmanaged_power']
    else:
        pltDF2 = rt_df[[f'{house}_total_power']]
        pltDF3 = rt_df[f'{house}_total_power']-rt_df[f'{house}_unmanaged_power']
    pltDF2 = pltDF2[PltStrtTime:PltEndTime]
    pltDF3 = pltDF3[PltStrtTime:PltEndTime]

    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12,12))
    pltDF.plot(ax=axes[0])
    pltDF2.plot(ax=axes[1])
    pltDF3.plot(ax=axes[2])
    axes[2].set(ylim=[max(-5,pltDF3.min()*1.05),min(10,pltDF3.max()*1.05)])

In [ ]:
#Plotting the Price to TMM for the entire day.
if use_TMM:
    pltDF = det_df[['TMM_1_price', 'DA ISO Energy Price']]
    pltDF.rename(columns={'TMM_1_price': 'Price to TMM'}, inplace=True)
    pltDF.plot(drawstyle="steps-post")

In [ ]:
DA_TMM_keys = [key for key in da_df if 'TMM' in key] #[]
print(f'DA Keys = {DA_TMM_keys}')
DET_TMM_keys = [key for key in det_df if 'TMM' in key] #['TMM_1_P', 'TMM_1_Q', 'TMM_1_price']
print(f'DET Keys = {DET_TMM_keys}')
RT_TMM_keys = [key for key in rt_df if 'TMM' in key] #['TMM_1_P_MPC', 'TMM_1_P_max', 'TMM_1_P_min', 'TMM_1_Rup_MPC', 'TMM_1_Rdn_MPC', 'TMM_1_price', 'TMM_1_P_setpt', 'Price to TMM', 'TMM_unmanaged_power', 'TMM_total_power']
print(f'RT Keys = {RT_TMM_keys}')

In [ ]:
[print(k) for k in det_df.keys()]

In [ ]:
df = pd.DataFrame()
df['max_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').max()
df['min_load_fcast'] = da_df.loc[:]['total_load'].groupby('time').min()


pltDF = pd.concat([rt_df[['Subs_p0_ISO','Subs_p0', 'MPC_dispatch', 'total_dispatch', 'total_dispatch_corrected','MPC_Rup','MPC_Rdn']], load_profile['load kW'], da_df.loc[1]['total_load'], df['max_load_fcast'], df['min_load_fcast']], axis = 1)
pltDF = pltDF.ffill()
pltDF = pltDF[PltStrtTime:PltEndTime]

fig, ax = plt.subplots(2, figsize = (10,10), sharex=True)
#plot the total dispatch and the substation power vs. 
ax[0].fill_between(pltDF.index,pltDF['max_load_fcast'],pltDF['min_load_fcast'],step='post',alpha=0.1,label='Load Forecasts Range across Scenarios', color = 'tab:blue')
ax[0].step(pltDF.index,pltDF['load kW'], where='post',label='Actual Load', color = 'tab:blue')
ax[0].step(pltDF.index,pltDF['Subs_p0_ISO'], where='post',label='ISO Dispatch', color = 'tab:red')
ax[0].plot(pltDF.index,pltDF['Subs_p0'],label='Measured Substation Power', linestyle='--', color = 'tab:orange')

ax[0].legend()
ax[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

ax[0].set (ylabel='Power [kW]', title = 'ISO Dispatch vs. Substation Power recorded in Simulation')

#plot the total dispatch and the substation power vs. 
ax[1].step(pltDF.index,pltDF['MPC_dispatch'], where='post',label='MPC Dispatch', color = 'tab:green')
ax[1].step(pltDF.index,pltDF['total_dispatch'], where='post',label='Actual Dispatch', color = 'tab:orange')
ax[1].fill_between(pltDF.index,pltDF['MPC_dispatch']+pltDF['MPC_Rup'],pltDF['MPC_dispatch']-pltDF['MPC_Rdn'],step='post',alpha=0.1,label='MPC Reserves', color = 'tab:green')
#ax[1].plot(pltDF.index,pltDF['total_dispatch_corrected'],label='Actual Dispatch Corrected', linestyle='--')

ax[1].legend()
ax[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)

ax[1].set (xlabel ='Time of Day', ylabel='Power [kW]', title = 'DER Power output')

ax[1].xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
ax[1].xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
ax[1].xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
ax[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

In [ ]:
# Percent Error in Dispatch:
pltDF['Percent Error'] = np.abs((pltDF['Subs_p0_ISO']-pltDF['Subs_p0'])/pltDF['Subs_p0_ISO']*100)

plot = pltDF['Percent Error'].plot(figsize = (10,6), title = 'Absolute Dispatch Percent Error', xlabel = 'Day of Month and Time of Day', ylabel = 'Absolute Error [%]')
plot.grid()
print(f' Average Dispatch error for the simulation is: {pltDF["Percent Error"].mean()}%')


In [ ]:
pltDF = rt_df[[f'{batt}_P' for batt in logs.BATTs]]
pltDF = pltDF[PltStrtTime:PltEndTime]

plot = pltDF.plot(figsize = (10,6))
#plot = RT_DF['ISO_Dispatch'].plot()
plot.set(xlabel = 'Day of Month and Time of Day', ylabel='Power [kW]', title = 'Measured DER Power Output on Simulation')
plot.grid()


In [ ]:
pltDF = rt_df[[f'{batt}_P_setpt' for batt in logs.BATTs]]
pltDF = pltDF[PltStrtTime:PltEndTime]

plot = pltDF.plot(figsize = (10,6))
#plot = RT_DF['ISO_Dispatch'].plot()
plot.set(xlabel = 'Day of Month and Time of Day', ylabel='Power [kW]', title = 'Measured DER Power Output on Simulation')
plot.grid()

In [ ]:
for ind, der in enumerate(logs.BATTs):
    rt_df[f'{der}_P_gen_conv'] = -rt_df[f'{der}_P']
    # Batt Data
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][der]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][der]['smax']/1000
    except:
        P_min = -P_max

    pltDF = pd.concat([rt_df[[f'{der}_P_MPC',f'{der}_P_setpt', f'{der}_P_gen_conv', f'{der}_P_max', f'{der}_P_min']]], axis = 1)
    pltDF = pltDF.ffill()
    pltDF = pltDF[PltStrtTime:PltEndTime]
    x0 = pltDF.index[0]
    x1 = pltDF.index[-1]

    fig, ax = plt.subplots(1, figsize = (10,6), sharex=True)
    ax.hlines(P_max,x0,x1,linestyles='dashed', color = 'grey', label='Min / Max')
    ax.hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    local_color = f'C{ind}'
    ax.fill_between(pltDF.index,pltDF[f'{der}_P_max'],pltDF[f'{der}_P_min'],step='post',alpha=0.1,label='MPC Reserves', color = local_color)
    ax.plot(pltDF.index,pltDF[f'{der}_P_MPC'],label='MPC', color = local_color, alpha = 0.8)
    ax.plot(pltDF.index,pltDF[f'{der}_P_setpt'],label='Setpoint', color = local_color, alpha = 0.5)
    ax.plot(pltDF.index,pltDF[f'{der}_P_gen_conv'],label='Measurement', color = local_color, linestyle=':')
   
    ax.legend()
    ax.grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax.set(xlabel='Time of Day', ylabel='Power [kW]', title = f'Battery Power of {der}')

    ax.xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
    ax.xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax.xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
    ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

In [ ]:
for ind, der in enumerate(logs.BATTs):
    rt_df[f'{der}_P_gen_conv'] = -rt_df[f'{der}_P']
    # Batt Data
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][der]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][der]['smax']/1000
    except:
        P_min = -P_max

    pltDF = pd.concat([rt_df[[f'{der}_P_MPC',f'{der}_P_setpt', f'{der}_P_gen_conv', f'{der}_P_max', f'{der}_P_min']], da_df.loc[1][f'{der}_P']], axis = 1)
    pltDF = pltDF.ffill()
    pltDF = pltDF[PltStrtTime:PltEndTime]
    x0 = pltDF.index[0]
    x1 = pltDF.index[-1]

    fig, ax = plt.subplots(1, figsize = (10,6), sharex=True)
    ax.hlines(P_max,x0,x1,linestyles='dashed', color = 'grey', label='Min / Max')
    ax.hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    local_color = f'C{ind}'
    ax.step(pltDF.index,pltDF[f'{der}_P'], where='post',label='DA', color =local_color, linestyle='--')
    ax.fill_between(pltDF.index,pltDF[f'{der}_P_max'],pltDF[f'{der}_P_min'],step='post',alpha=0.1,label='MPC Reserves', color = local_color)
    ax.plot(pltDF.index,pltDF[f'{der}_P_MPC'],label='MPC', color = local_color, alpha = 0.8)
    ax.plot(pltDF.index,pltDF[f'{der}_P_setpt'],label='Setpoint', color = local_color, alpha = 0.5)
    ax.plot(pltDF.index,pltDF[f'{der}_P_gen_conv'],label='Measurement', color = local_color, linestyle=':')
   
    ax.legend()
    ax.grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax.set(xlabel='Time of Day', ylabel='Power [kW]', title = f'Battery Power of {der}')

    ax.xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
    ax.xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax.xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
    ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

In [ ]:
for ind, der in enumerate(logs.BATTs):
    rt_df[f'{der}_P_gen_conv'] = -rt_df[f'{der}_P']
    # Batt Data
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][der]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][der]['smax']/1000
    except:
        P_min = -P_max

    E_max = 0.9
    E_min = 0.1

    pltDF = pd.concat([rt_df[[f'{der}_P_MPC',f'{der}_P_setpt', f'{der}_P_gen_conv', f'{der}_P_max', f'{der}_P_min', f'{der}_soc']]], axis = 1)
    pltDF = pltDF.ffill()
    pltDF = pltDF[PltStrtTime:PltEndTime]
    x0 = pltDF.index[0]
    x1 = pltDF.index[-1]

    fig, ax = plt.subplots(2, figsize = (10,6), sharex=True)
    ax[0].hlines(P_max,x0,x1,linestyles='dashed', color = 'grey', label='Min / Max')
    ax[0].hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    local_color = f'C{ind}'
    ax[0].fill_between(pltDF.index,pltDF[f'{der}_P_max'],pltDF[f'{der}_P_min'],step='post',alpha=0.1,label='MPC Reserves', color = local_color)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_MPC'],label='MPC', color = local_color, alpha = 0.8)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_setpt'],label='Setpoint', color = local_color, alpha = 0.5)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_gen_conv'],label='Measurement', color = local_color, linestyle=':')
   
    ax[0].legend()
    ax[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax[0].set(ylabel='Power [kW]', title = f'Battery Power of {der}')

    ax[1].hlines(E_max,x0,x1,linestyles='dashed', color = 'grey', label='Min / Max')
    ax[1].hlines(E_min,x0,x1,linestyles='dashed', color = 'grey')
    ax[1].plot(pltDF.index,pltDF[f'{der}_soc'],label='Measurement', color = local_color, linestyle=':')
    ax[1].legend()
    ax[1].set(xlabel = 'Time of Day', ylabel='SoC [-]', title = f'Battery SoC of {der}')
    ax[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax[1].xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
    ax[1].xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax[1].xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
    ax[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

In [ ]:
for ind, der in enumerate(logs.BATTs):
    rt_df[f'{der}_P_gen_conv'] = -rt_df[f'{der}_P']
    rt_df[f'{der}_soc_MPC'] = rt_df[f'{der}_E_MPC']/(logs.static_data['DERs'][der]['emax']/1000)
    # Batt Data
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][der]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][der]['smax']/1000
    except:
        P_min = -P_max

    E_max = 0.9
    E_min = 0.1

    pltDF = pd.concat([rt_df[[f'{der}_P_MPC',f'{der}_P_setpt', f'{der}_P_gen_conv', f'{der}_P_max', f'{der}_P_min', f'{der}_soc', f'{der}_soc_MPC']]], axis = 1)
    pltDF = pltDF.ffill()
    pltDF = pltDF[PltStrtTime:PltEndTime]
    x0 = pltDF.index[0]
    x1 = pltDF.index[-1]

    fig, ax = plt.subplots(2, figsize = (10,6), sharex=True)
    ax[0].hlines(P_max,x0,x1,linestyles='dashed', color = 'grey', label='Min / Max')
    ax[0].hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    local_color = f'C{ind}'
    ax[0].fill_between(pltDF.index,pltDF[f'{der}_P_max'],pltDF[f'{der}_P_min'],step='post',alpha=0.1,label='MPC Reserves', color = local_color)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_MPC'],label='MPC', color = local_color, alpha = 0.8)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_setpt'],label='Setpoint', color = local_color, alpha = 0.5)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_gen_conv'],label='Measurement', color = local_color, linestyle=':')
   
    ax[0].legend()
    ax[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax[0].set(ylabel='Power [kW]', title = f'Battery Power of {der}')
    ax[1].hlines(E_max,x0,x1,linestyles='dashed', color = 'grey', label='Min / Max')
    ax[1].hlines(E_min,x0,x1,linestyles='dashed', color = 'grey')
    ax[1].plot(pltDF.index,pltDF[f'{der}_soc_MPC'],label='MPC', color = local_color)
    ax[1].plot(pltDF.index,pltDF[f'{der}_soc'],label='Measurement', color = local_color, linestyle=':')
    ax[1].legend()
    ax[1].set(xlabel = 'Time of Day', ylabel='Power [kW]', title = f'Battery Energy of {der}')
    ax[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax[1].xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
    ax[1].xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax[1].xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
    ax[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

In [ ]:
for ind, der in enumerate(logs.BATTs):
    rt_df[f'{der}_P_gen_conv'] = -rt_df[f'{der}_P']
    rt_df[f'{der}_soc_MPC'] = rt_df[f'{der}_E_MPC']/(logs.static_data['DERs'][der]['emax']/1000)
    # Batt Data
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][der]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][der]['smax']/1000
    except:
        P_min = -P_max

    E_max = 0.9
    E_min = 0.1

    pltDF = pd.concat([rt_df[[f'{der}_P_MPC',f'{der}_P_setpt', f'{der}_P_gen_conv', f'{der}_P_max', f'{der}_P_min', f'{der}_soc', f'{der}_soc_MPC']], da_df.loc[1][f'{der}_P']], axis = 1)
    pltDF = pltDF.ffill()
    pltDF = pltDF[PltStrtTime:PltEndTime]
    x0 = pltDF.index[0]
    x1 = pltDF.index[-1]

    fig, ax = plt.subplots(2, figsize = (10,6), sharex=True)
    ax[0].hlines(P_max,x0,x1,linestyles='dashed', color = 'grey', label='Min / Max')
    ax[0].hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    local_color = f'C{ind}'
    ax[0].step(pltDF.index,pltDF[f'{der}_P'], where='post',label='DA', color = local_color, linestyle='--')
    ax[0].fill_between(pltDF.index,pltDF[f'{der}_P_max'],pltDF[f'{der}_P_min'],step='post',alpha=0.1,label='MPC Reserves', color = local_color)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_MPC'],label='MPC', color = local_color, alpha = 0.8)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_setpt'],label='Setpoint', color = local_color, alpha = 0.5)
    ax[0].plot(pltDF.index,pltDF[f'{der}_P_gen_conv'],label='Measurement', color = local_color, linestyle=':')
   
    ax[0].legend()
    ax[0].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax[0].set(ylabel='Power [kW]', title = f'Battery Power of {der}')
    ax[1].hlines(E_max,x0,x1,linestyles='dashed', color = 'grey', label='Min / Max')
    ax[1].hlines(E_min,x0,x1,linestyles='dashed', color = 'grey')
    ax[1].plot(pltDF.index,pltDF[f'{der}_soc_MPC'],label='MPC', color = local_color)
    ax[1].plot(pltDF.index,pltDF[f'{der}_soc'],label='Measurement', color = local_color, linestyle=':')
    ax[1].legend()
    ax[1].set(xlabel = 'Time of Day', ylabel='Power [kW]', title = f'Battery Energy of {der}')
    ax[1].grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax[1].xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
    ax[1].xaxis.set_minor_formatter(dates.DateFormatter(':%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax[1].xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
    ax[1].xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

# Paper Figures

In [ ]:
for ind, der in enumerate(logs.BATTs):
    # Batt data
    x0 = da_df.loc[1].index[0]
    x1 = da_df.loc[1].index[-1]
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][der]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][der]['smax']/1000
    except:
        P_min = -P_max

    batt_p_rup = da_df.loc[1][f'{der}_P'] + da_df.loc[1][f'{der}_Rup']
    batt_p_rdn = da_df.loc[1][f'{der}_P'] - da_df.loc[1][f'{der}_Rdn']

    fig, ax = plt.subplots(1, figsize = (10,6), sharex=True)
    ax.hlines(P_max,x0,x1,linestyles='dashed', color = 'grey')
    ax.hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    ax.step(da_df.loc[1].index,da_df.loc[1][f'{der}_P'], where='post',label='DA', color = f'C{ind}')
    ax.fill_between(da_df.loc[1].index,batt_p_rup,batt_p_rdn,step='post',color = f'C{ind}', alpha=0.3,label='Reserves')
    ax.plot(rt_df.index,rt_df[f'{der}_P_MPC'],label='MPC', color = 'red')
    ax.plot(rt_df.index,rt_df[f'{der}_P_setpt'],label='Setpoint', color = 'blue')
    ax.plot(rt_df.index,rt_df[f'{der}_P_gen_conv'],label='Measurement', color = 'black')
   
    ax.legend()
    ax.set_xlim([PltStrtTime, PltEndTime])
    ax.set(ylabel='Power [kW]', title = f'Battery Power of {der}')
    ax.grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
   
    ax.set(xlabel = 'Time of Day', ylabel='Power [kW]', title = f'Battery Power of {der}')
    ax.grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax.xaxis.set_minor_locator(dates.MinuteLocator(interval=15))   
    ax.xaxis.set_minor_formatter(dates.DateFormatter('', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax.xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
    ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes

In [ ]:

for ind, der in enumerate(logs.BATTs):
    # Batt data
    x0 = da_df.loc[1].index[0]
    x1 = da_df.loc[1].index[-1]
    ## Pmin / Pmax 
    P_max = logs.static_data['DERs'][der]['emax']/1000*0.8*0.5
    try:
        P_min = -logs.static_data['DERs'][der]['smax']/1000
    except:
        P_min = -P_max

    df = pd.DataFrame()
    df['max_p'] = da_df.loc[:][f'{der}_P'].groupby('time').max()
    df['min_p'] = da_df.loc[:][f'{der}_P'].groupby('time').min()

    pltDF = pd.concat([rt_df[[f'{der}_P_gen_conv',f'{der}_P_setpt', f'{der}_P_MPC']], da_df.loc[1][f'{der}_P'], df['max_p'], df['min_p']], axis = 1)
    pltDF = pltDF.ffill()
    pltDF = pltDF[PltStrtTime:PltEndTime]       

    fig, ax = plt.subplots(1, figsize = (10,6), sharex=True)
    ax.hlines(P_max,x0,x1,linestyles='dashed', color = 'grey')
    ax.hlines(P_min,x0,x1,linestyles='dashed', color = 'grey')
    ax.fill_between(pltDF.index,pltDF['max_p'],pltDF['min_p'],step='post',color = f'C{ind}', alpha=0.3,label='Range across Scenarios')
    ax.step(pltDF.index,pltDF[f'{der}_P'], where='post',label='DA', color = f'C{ind}')
    ax.plot(pltDF.index,pltDF[f'{der}_P_MPC'],label='MPC', color = 'red')
    ax.plot(pltDF.index,pltDF[f'{der}_P_setpt'],label='Setpoint', color = 'blue')
    ax.plot(pltDF.index,pltDF[f'{der}_P_gen_conv'],label='Measurement', color = 'black')
   
    ax.legend()
    ax.set_xlim([PltStrtTime, PltEndTime])
    ax.set(ylabel='Power [kW]', title = f'Battery Power of {der}')
    ax.grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
   
    ax.set(xlabel = 'Time of Day', ylabel='Power [kW]', title = f'Battery Power of {der}')
    ax.grid(True, which='both', axis='both', color='gray', linestyle='--', linewidth=0.5)
    ax.xaxis.set_minor_locator(dates.MinuteLocator(byminute=range(0,60,15)))   
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    ax.xaxis.set_major_locator(dates.HourLocator(interval=1))    # every 1 hours
    ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M', tz=da_df.loc[1].index.tz))  # hours and minutes
    
    df = pd.DataFrame()




# Understanding the DET

In [ ]:
[print(ky) for ky in det_df.keys()]